#!pip install pmdarima

In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
import datetime
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
#from pmdarima import auto_arima
import warnings
from sklearn.metrics import mean_absolute_percentage_error
# import os
# 
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
# 
# warnings.filterwarnings("ignore") # Импорт необходимых библиотек.


In [2]:
df= pd.read_csv('./data/train_final.csv')
df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date
0,City Hotel,0,25,2017,February,8,22,0,4,3,...,E,E,0,No Deposit,0,Transient,124.10,0,1,2017-02-26
1,City Hotel,0,134,2016,August,32,1,1,4,2,...,F,F,0,No Deposit,0,Transient,176.45,0,1,2016-08-06
2,Resort Hotel,0,0,2015,November,48,24,0,1,1,...,A,A,0,No Deposit,0,Transient,26.00,1,0,2015-11-25
3,Resort Hotel,0,86,2015,October,43,18,2,5,2,...,D,D,0,No Deposit,0,Transient,50.03,0,0,2015-10-25
4,City Hotel,0,156,2016,July,29,11,2,5,2,...,A,D,1,No Deposit,0,Transient,94.91,0,1,2016-07-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44643,City Hotel,1,265,2017,August,31,5,3,6,2,...,A,A,0,No Deposit,0,Transient,89.10,0,0,2016-11-13
44644,City Hotel,1,6,2016,August,33,8,1,0,1,...,A,A,0,No Deposit,0,Transient-Party,65.00,0,0,2016-08-08
44645,Resort Hotel,1,106,2016,April,17,23,2,2,2,...,A,A,0,No Deposit,0,Transient,52.20,0,0,2016-04-21
44646,City Hotel,1,151,2017,May,21,27,1,1,2,...,A,A,0,No Deposit,0,Transient,133.00,0,0,2016-12-27


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44648 entries, 0 to 44647
Data columns (total 29 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           44648 non-null  object 
 1   is_canceled                     44648 non-null  int64  
 2   lead_time                       44648 non-null  int64  
 3   arrival_date_year               44648 non-null  int64  
 4   arrival_date_month              44648 non-null  object 
 5   arrival_date_week_number        44648 non-null  int64  
 6   arrival_date_day_of_month       44648 non-null  int64  
 7   stays_in_weekend_nights         44648 non-null  int64  
 8   stays_in_week_nights            44648 non-null  int64  
 9   adults                          44648 non-null  int64  
 10  children                        44648 non-null  float64
 11  babies                          44648 non-null  int64  
 12  meal                            

In [4]:
df.nunique()

hotel                                2
is_canceled                          2
lead_time                          467
arrival_date_year                    3
arrival_date_month                  12
arrival_date_week_number            53
arrival_date_day_of_month           31
stays_in_weekend_nights             14
stays_in_week_nights                28
adults                               7
children                             5
babies                               4
meal                                 5
country                            157
market_segment                       7
distribution_channel                 4
is_repeated_guest                    2
previous_cancellations              14
previous_bookings_not_canceled      53
reserved_room_type                  10
assigned_room_type                  11
booking_changes                     17
deposit_type                         3
days_in_waiting_list               112
customer_type                        4
adr                      

In [5]:
df.drop(columns=['arrival_date_week_number'],
        inplace=True)

In [6]:
df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,reserved_room_type,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date
0,City Hotel,0,25,2017,February,22,0,4,3,0.0,...,E,E,0,No Deposit,0,Transient,124.10,0,1,2017-02-26
1,City Hotel,0,134,2016,August,1,1,4,2,2.0,...,F,F,0,No Deposit,0,Transient,176.45,0,1,2016-08-06
2,Resort Hotel,0,0,2015,November,24,0,1,1,0.0,...,A,A,0,No Deposit,0,Transient,26.00,1,0,2015-11-25
3,Resort Hotel,0,86,2015,October,18,2,5,2,0.0,...,D,D,0,No Deposit,0,Transient,50.03,0,0,2015-10-25
4,City Hotel,0,156,2016,July,11,2,5,2,1.0,...,A,D,1,No Deposit,0,Transient,94.91,0,1,2016-07-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44643,City Hotel,1,265,2017,August,5,3,6,2,0.0,...,A,A,0,No Deposit,0,Transient,89.10,0,0,2016-11-13
44644,City Hotel,1,6,2016,August,8,1,0,1,0.0,...,A,A,0,No Deposit,0,Transient-Party,65.00,0,0,2016-08-08
44645,Resort Hotel,1,106,2016,April,23,2,2,2,0.0,...,A,A,0,No Deposit,0,Transient,52.20,0,0,2016-04-21
44646,City Hotel,1,151,2017,May,27,1,1,2,0.0,...,A,A,0,No Deposit,0,Transient,133.00,0,0,2016-12-27


In [ ]:
df_date= df['arrival_date_month'].str[0:3]
month_number = np.zeros(len(df_date))

In [9]:
df['arrival_data']=df['arrival_date_year'].map(str) + '-' + df['arrival_date_month'].map(str) + '-'  + df['arrival_date_day_of_month'].map(str)

In [10]:
df

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,...,assigned_room_type,booking_changes,deposit_type,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_date,arrival_data
0,City Hotel,0,25,2017,February,22,0,4,3,0.0,...,E,0,No Deposit,0,Transient,124.10,0,1,2017-02-26,2017-February-22
1,City Hotel,0,134,2016,August,1,1,4,2,2.0,...,F,0,No Deposit,0,Transient,176.45,0,1,2016-08-06,2016-August-1
2,Resort Hotel,0,0,2015,November,24,0,1,1,0.0,...,A,0,No Deposit,0,Transient,26.00,1,0,2015-11-25,2015-November-24
3,Resort Hotel,0,86,2015,October,18,2,5,2,0.0,...,D,0,No Deposit,0,Transient,50.03,0,0,2015-10-25,2015-October-18
4,City Hotel,0,156,2016,July,11,2,5,2,1.0,...,D,1,No Deposit,0,Transient,94.91,0,1,2016-07-18,2016-July-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44643,City Hotel,1,265,2017,August,5,3,6,2,0.0,...,A,0,No Deposit,0,Transient,89.10,0,0,2016-11-13,2017-August-5
44644,City Hotel,1,6,2016,August,8,1,0,1,0.0,...,A,0,No Deposit,0,Transient-Party,65.00,0,0,2016-08-08,2016-August-8
44645,Resort Hotel,1,106,2016,April,23,2,2,2,0.0,...,A,0,No Deposit,0,Transient,52.20,0,0,2016-04-21,2016-April-23
44646,City Hotel,1,151,2017,May,27,1,1,2,0.0,...,A,0,No Deposit,0,Transient,133.00,0,0,2016-12-27,2017-May-27
